In [15]:
from api.db_models.weaviate_db import WeaviateIndustryDb
import json

import weaviate
client = weaviate.connect_to_local(
    port=8002,
    skip_init_checks=True
)

client.collections.delete('IndustrySummary')

In [11]:
filename = 'IndustrySummary/IBIS/Furniture Stores in the US.json'
summary = json.load(open(filename, 'r'))
db = WeaviateIndustryDb(url="http://localhost:8002")

[m for m in dir(db.client) if not m.startswith('_')]
# db.client.collections.delete('IndustrySummary')
# db.add_industry_summary(summary)

/tmp/ipykernel_68915/1220338876.py:2: ResourceWarning: unclosed file <_io.TextIOWrapper name='IndustrySummary/IBIS/Furniture Stores in the US.json' mode='r' encoding='UTF-8'>
  summary = json.load(open(filename, 'r'))
/home/yakov/anaconda3/envs/probe/lib/python3.11/site-packages/weaviate/warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


['backup',
 'batch',
 'classification',
 'cluster',
 'contextionary',
 'data_object',
 'get_meta',
 'get_open_id_configuration',
 'is_live',
 'is_ready',
 'query',
 'schema',
 'timeout_config']

In [ ]:
from dateparser import parse
dd = parse('October 23, 1995')
str(dd)
dd.isoformat()

In [ ]:
import datetime

n = datetime.datetime.now(datetime.timezone.utc)
n.isoformat() # '2021-07-13T15:28:51.818095+00:00'